# Rema1000 Discount Crawler

## Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

## Crawler

In [2]:
time_wait = 3

In [3]:
url = "https://shop.rema1000.dk/"
driver = webdriver.Firefox()
driver.get(url)

# accept cookies
WebDriverWait(driver, time_wait).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#declineButton"))).click()

# wait and refresh to load correct page content
driver.refresh()
time.sleep(time_wait)
soup = BeautifulSoup(driver.page_source)

# get all urls
items = soup.find_all("a", class_="item")

urls = []

for item in items:
    urls.append(str(item).split('href="')[1].split('"')[0])

urls = [url[:-1] + item for item in urls]

# create an empty dataframe
all_products = pd.DataFrame()
columns = ['product_title', 'product_price', 'product_price_per_unit', 'product_subtitle', 'product_weight', 'product_brand', 'product_discount', 'date']
all_products = pd.DataFrame(columns=columns)

# go through each category and get all products
sub_category_urls = []

for category_url in urls:
    driver.get(category_url)
    time.sleep(time_wait)
    soup = BeautifulSoup(driver.page_source)

    sub_categories = soup.find_all("a", class_="all")

    for sub_category in sub_categories:
        sub_category_urls.append((str(sub_category).split('href="')[1].split('"')[0]))

# make sub_category_urls complete
sub_category_urls = [url[:-1] + sub_category_url for sub_category_url in sub_category_urls]

# go through each subcategory and get all products
for sub_category_url in sub_category_urls:
    # load pages
    driver.get(sub_category_url)
    time.sleep(time_wait) 
    soup = BeautifulSoup(driver.page_source)

    # get all products
    products = soup.find_all("a", class_="product-wrap")

    # create an empty dataframe
    product_category = pd.DataFrame()
    columns = ['product_title', 'product_price', 'product_price_per_unit', 'product_subtitle', 'product_weight', 'product_brand', 'product_discount', 'date']
    product_category = pd.DataFrame(columns=columns)

    # go through each product
    for product in products:
        current_product = []
        current_product.append(str(product.find("div", class_="title")).split('"">')[1].split('</div>')[0])
        current_product.append(str(product.find("div", class_="price").find("span", class_="price-normal")).split('"">')[1].split('</')[0].replace("<span>","."))
        current_product.append(str(product.find("div", class_="price").find("span", class_="price-per-unit")).split('"">')[1].split('</')[0])
        current_product.append(str(product.find("div", class_="extra")).split('"">')[2].split("</")[0])
        current_product.append(str(product.find("div", class_="extra")).split('"">')[2].split("</")[0].split(" / ")[0])
        current_product.append(str(product.find("div", class_="extra")).split('"">')[2].split("</")[0].split(" / ")[1])
        current_product.append(1 if "avisvare" in str(product.find_all("div", class_="top")) else 0)
        current_product.append(datetime.datetime.now().strftime("%Y-%m-%d"))

        # product df
        current_product = pd.DataFrame(current_product).transpose()
        current_product.columns = columns

        # add product df to category 
        product_category = pd.concat([product_category, current_product], ignore_index=True)

    # add product category to all products 
    all_products = pd.concat([all_products, product_category], ignore_index=True)
    
driver.quit()

# v1 RUNTIME: 2m 17s
# v2 RUNTIME: 12m 26s
# v3 RUNTIME: 11m 20s

In [4]:
print(len(sub_category_urls)) # 158
print(sub_category_urls)

158
['https://shop.rema1000.dk/brod-bavinchi/baguetteflutes', 'https://shop.rema1000.dk/brod-bavinchi/boller', 'https://shop.rema1000.dk/brod-bavinchi/brod', 'https://shop.rema1000.dk/brod-bavinchi/rugbrod', 'https://shop.rema1000.dk/brod-bavinchi/fast-food-brod', 'https://shop.rema1000.dk/brod-bavinchi/bavinchi-bager', 'https://shop.rema1000.dk/frugt-gront/paere-aeble-banan-citrusfrugt', 'https://shop.rema1000.dk/frugt-gront/blommer-fersken-nektariner-kiwi', 'https://shop.rema1000.dk/frugt-gront/melon-baer-vindruer-eksotisk-frugt', 'https://shop.rema1000.dk/frugt-gront/agurk-tomat-peberfrugt', 'https://shop.rema1000.dk/frugt-gront/salater-fint-gront-avocado', 'https://shop.rema1000.dk/frugt-gront/krydderurter-smagsforstaerkere', 'https://shop.rema1000.dk/frugt-gront/grov-gront', 'https://shop.rema1000.dk/frugt-gront/kal', 'https://shop.rema1000.dk/frugt-gront/kartofler-log', 'https://shop.rema1000.dk/nemt-hurtigt/snittet-frugt-gront', 'https://shop.rema1000.dk/nemt-hurtigt/maltidssala

## Stats

In [5]:
print(f"products scraped: {all_products.shape[0]}") # 3662
print(f"unique products scraped: {all_products['product_title'].nunique()}") # 2858
print(f"products with discount: {all_products[all_products['product_discount'] == 1].shape[0]}") # 408: correct

products scraped: 3662
unique products scraped: 2858
products with discount: 408


## Inspect data

In [6]:
all_products

,product_title,product_price,product_price_per_unit,product_subtitle,product_weight,product_brand,product_discount,date
0,FLUTES,6.50,21.67 per Kg.,300 GR. / REMA 1000,300 GR.,REMA 1000,0,2024-01-13
1,CIABATTA,6.95,23.17 per Kg.,300 GR. / REMA 1000,300 GR.,REMA 1000,0,2024-01-13
2,FULDKORNSFLUTES,12.00,40.00 per Kg.,300 GR. / REMA 1000,300 GR.,REMA 1000,0,2024-01-13
3,SOLSIKKEBOLLER,25.50,51.00 per Kg.,500 GR. / DET GODE,500 GR.,DET GODE,0,2024-01-13
4,KRYDDERBOLLER,8.50,24.29 per Kg.,350 GR. / REMA 1000,350 GR.,REMA 1000,0,2024-01-13
...,...,...,...,...,...,...,...,...
3657,TYRKISK PEBER 32%,36.95,369.50 per Ltr.,10 CL. / HOT N´SWEET,10 CL.,HOT N´SWEET,0,2024-01-13
3658,"SMÅ FUGLE 16,4%",99.00,99.00 per Ltr.,1 LTR. /,1 LTR.,,0,2024-01-13
3659,"SOUR BUBBLE FIZZ 16,4%",99.00,99.00 per Ltr.,1 LTR. / SMÅ SURE,1 LTR.,SMÅ SURE,0,2024-01-13
3660,EXTRA GAS,12.50,41.67 per Ltr.,300 ML. / LIGHTER,300 ML.,LIGHTER,0,2024-01-13


In [7]:
all_products[all_products['product_discount'] == 1]

,product_title,product_price,product_price_per_unit,product_subtitle,product_weight,product_brand,product_discount,date
5,MULTIKERNEBOLLER,12.00,21.82 per Kg.,550 GR. / LEVEBRØD,550 GR.,LEVEBRØD,1,2024-01-13
7,CHIA BOLLER,12.00,23.08 per Kg.,520 GR. / LEVEBRØD,520 GR.,LEVEBRØD,1,2024-01-13
12,FULDKORNS BOLLER,12.00,25.53 per Kg.,470 GR. / LEVEBRØD,470 GR.,LEVEBRØD,1,2024-01-13
22,MULTIK. SANDWICH,12.00,15.00 per Kg.,800 GR. / LEVEBRØD,800 GR.,LEVEBRØD,1,2024-01-13
33,KERNEGROVBRØD,12.00,12.63 per Kg.,950 GR. / LEVEBRØD,950 GR.,LEVEBRØD,1,2024-01-13
...,...,...,...,...,...,...,...,...
3585,P-TÆRTER,10.00,142.86 per Kg.,"70 GR. / CARLETTI, KARAMEL",70 GR.,"CARLETTI, KARAMEL",1,2024-01-13
3609,ENGELSK VINGUMMI,20.00,80.00 per Kg.,250 GR. / REMA 1000,250 GR.,REMA 1000,1,2024-01-13
3610,SUR MIX,20.00,85.11 per Kg.,235 GR. / REMA 1000,235 GR.,REMA 1000,1,2024-01-13
3611,VINGUMMIBAMSER,20.00,100.00 per Kg.,200 GR. / REMA 1000,200 GR.,REMA 1000,1,2024-01-13


## Save data as CSV

In [10]:
# Get current yearmonth
now = datetime.datetime.now()
week = now.strftime("%U")
year = now.strftime("%Y")

# Save to excel in data folder
all_products.to_excel(f"data/rema_W{week}_{year}.xlsx")

# Keep a consolidated file with all data in root folder

In [ ]:
# Calculate the discount
# 1. Find products currently on discount (filter discount == 1)
# 2. Find the latest price for the discounted product (sort(date))
# 3. Calculate the discount (price - latest_price) / latest_price
# 4. Add the discount to a new dataframe
# 5. Flag fake discounts? (price == latest_price)
# 6. Track inflation?

End of document